## DIA 125: Agregar Nuevas Categorías desde la Interfaz al Subir un Reporte

Permitir al usuario agregar nuevas categorías personalizadas desde el formulario de carga de reportes, sin necesidad de escribir manualmente, usando un campo que combine un menú desplegable de categorías existentes con la opción de escribir una nueva.

🧩 1. Vista cargar_reporte en Flask
Se modificará para:

Obtener las categorías existentes.

Procesar una categoría nueva si es proporcionada.

python
Copiar
Editar
@app.route('/cargar', methods=['GET', 'POST'])
@login_required
def cargar_reporte():
    categorias = db.session.query(Reporte.categoria).filter_by(user_id=current_user.id).distinct().all()
    categorias = [c[0] for c in categorias if c[0]]

    if request.method == 'POST':
        titulo = request.form['titulo']
        nueva_categoria = request.form.get('nueva_categoria', '').strip()
        categoria_seleccionada = request.form.get('categoria_existente', '').strip()

        categoria_final = nueva_categoria if nueva_categoria else categoria_seleccionada

        nuevo_reporte = Reporte(
            titulo=titulo,
            categoria=categoria_final,
            user_id=current_user.id
        )
        db.session.add(nuevo_reporte)
        db.session.commit()

        flash("Reporte cargado exitosamente.", "success")
        return redirect(url_for('historial_reportes'))

    return render_template("cargar_reporte.html", categorias=categorias)
🧩 2. Formulario HTML (cargar_reporte.html)
Formulario que permite:

Seleccionar una categoría existente.

Escribir una nueva categoría (opcional).

html
Copiar
Editar
<form method="POST" action="{{ url_for('cargar_reporte') }}">
  <label for="titulo">Título:</label>
  <input type="text" name="titulo" required><br>

  <label for="categoria_existente">Selecciona una categoría:</label>
  <select name="categoria_existente" id="categoria_existente">
    <option value="">-- Ninguna --</option>
    {% for cat in categorias %}
      <option value="{{ cat }}">{{ cat }}</option>
    {% endfor %}
  </select><br>

  <label for="nueva_categoria">O escribe una nueva categoría:</label>
  <input type="text" name="nueva_categoria" id="nueva_categoria"><br>

  <button type="submit">Cargar reporte</button>
</form>